#### Pseudocode for the poker simulation:

In [212]:
from collections import namedtuple
import itertools
 
class Card(namedtuple('Card', 'face, suit')):
    def __repr__(self):
        return ''.join(self)
 
 
suit = '♥ ♦ ♣ ♠'.split()
# ordered strings of faces
faces   = '2 3 4 5 6 7 8 9 10 j q k a'
lowaces = 'a 2 3 4 5 6 7 8 9 10 j q k'
deck = list(itertools.product(faces.split(), suit))
print(deck)
# faces as lists
face   = faces.split()
lowace = lowaces.split()
 
 
def straightflush(hand):
    f,fs = ( (lowace, lowaces) if any(card.face == '2' for card in hand)
             else (face, faces) )
    ordered = sorted(hand, key=lambda card: (f.index(card.face), card.suit))
    first, rest = ordered[0], ordered[1:]
    if ( all(card.suit == first.suit for card in rest) and
         ' '.join(card.face for card in ordered) in fs ):
        return 'straight-flush', ordered[-1].face
    return False
 
def fourofakind(hand):
    allfaces = [f for f,s in hand]
    allftypes = set(allfaces)
    if len(allftypes) != 2:
        return False
    for f in allftypes:
        if allfaces.count(f) == 4:
            allftypes.remove(f)
            return 'four-of-a-kind', [f, allftypes.pop()]
    else:
        return False
 
def fullhouse(hand):
    allfaces = [f for f,s in hand]
    allftypes = set(allfaces)
    if len(allftypes) != 2:
        return False
    for f in allftypes:
        if allfaces.count(f) == 3:
            allftypes.remove(f)
            return 'full-house', [f, allftypes.pop()]
    else:
        return False
 
def flush(hand):
    allstypes = {s for f, s in hand}
    if len(allstypes) == 1:
        allfaces = [f for f,s in hand]
        return 'flush', sorted(allfaces,
                               key=lambda f: face.index(f),
                               reverse=True)
    return False
 
def straight(hand):
    f,fs = ( (lowace, lowaces) if any(card.face == '2' for card in hand)
             else (face, faces) )
    ordered = sorted(hand, key=lambda card: (f.index(card.face), card.suit))
    first, rest = ordered[0], ordered[1:]
    if ' '.join(card.face for card in ordered) in fs:
        return 'straight', ordered[-1].face
    return False
 
def threeofakind(hand):
    allfaces = [f for f,s in hand]
    allftypes = set(allfaces)
    if len(allftypes) <= 2:
        return False
    for f in allftypes:
        if allfaces.count(f) == 3:
            allftypes.remove(f)
            return ('three-of-a-kind', [f] +
                     sorted(allftypes,
                            key=lambda f: face.index(f),
                            reverse=True))
    else:
        return False
 
def twopair(hand):
    allfaces = [f for f,s in hand]
    allftypes = set(allfaces)
    pairs = [f for f in allftypes if allfaces.count(f) == 2]
    if len(pairs) != 2:
        return False
    p0, p1 = pairs
    other = [(allftypes - set(pairs)).pop()]
    return 'two-pair', pairs + other if face.index(p0) > face.index(p1) else pairs[::-1] + other
 
def onepair(hand):
    allfaces = [f for f,s in hand]
    allftypes = set(allfaces)
    pairs = [f for f in allftypes if allfaces.count(f) == 2]
    if len(pairs) != 1:
        return False
    allftypes.remove(pairs[0])
    return 'one-pair', pairs + sorted(allftypes,
                                      key=lambda f: face.index(f),
                                      reverse=True)
 
def highcard(hand):
    allfaces = [f for f,s in hand]
    return 'high-card', sorted(allfaces,
                               key=lambda f: face.index(f),
                               reverse=True)
 
handrankorder =  (straightflush, fourofakind, fullhouse,
                  flush, straight, threeofakind,
                  twopair, onepair, highcard)
 
def rank(cards):
    hand = handy(cards)
    for ranker in handrankorder:
        rank = ranker(hand)
        if rank:
            break
    assert rank, "Invalid: Failed to rank cards: %r" % cards
    return rank
 
def handy(cards='2♥ 2♦ 2♣ k♣ q♦'):
    hand = []
    for card in cards.split():
        f, s = card[:-1], card[-1]
        assert f in face, "Invalid: Don't understand card face %r" % f
        assert s in suit, "Invalid: Don't understand card suit %r" % s
        hand.append(Card(f, s))
    assert len(hand) == 5, "Invalid: Must be 5 cards in a hand, not %i" % len(hand)
    assert len(set(hand)) == 5, "Invalid: All cards in the hand must be unique %r" % cards
    return hand
 
 
#if __name__ == '__main__':
#    hands = ["2♥ 2♦ 2♣ k♣ q♦",
#     "2♥ 5♥ 7♦ 8♣ 9♠",
#     "a♥ 2♦ 3♣ 4♣ 5♦",
#     "2♥ 3♥ 2♦ 3♣ 3♦",
#     "2♥ 7♥ 2♦ 3♣ 3♦",
#     "2♥ 7♥ 7♦ 7♣ 7♠",
#     "10♥ j♥ q♥ k♥ a♥"] + [
#     "4♥ 4♠ k♠ 5♦ 10♠",
#     "q♣ 10♣ 7♣ 6♣ 4♣",
#     ]
#    print("%-18s %-15s %s" % ("HAND", "CATEGORY", "TIE-BREAKER"))
#    for cards in hands:
#        r = rank(cards)
#        print("%-18r %-15s %r" % (cards, r[0], r[1]))

[('2', '♥'), ('2', '♦'), ('2', '♣'), ('2', '♠'), ('3', '♥'), ('3', '♦'), ('3', '♣'), ('3', '♠'), ('4', '♥'), ('4', '♦'), ('4', '♣'), ('4', '♠'), ('5', '♥'), ('5', '♦'), ('5', '♣'), ('5', '♠'), ('6', '♥'), ('6', '♦'), ('6', '♣'), ('6', '♠'), ('7', '♥'), ('7', '♦'), ('7', '♣'), ('7', '♠'), ('8', '♥'), ('8', '♦'), ('8', '♣'), ('8', '♠'), ('9', '♥'), ('9', '♦'), ('9', '♣'), ('9', '♠'), ('10', '♥'), ('10', '♦'), ('10', '♣'), ('10', '♠'), ('j', '♥'), ('j', '♦'), ('j', '♣'), ('j', '♠'), ('q', '♥'), ('q', '♦'), ('q', '♣'), ('q', '♠'), ('k', '♥'), ('k', '♦'), ('k', '♣'), ('k', '♠'), ('a', '♥'), ('a', '♦'), ('a', '♣'), ('a', '♠')]


In [213]:
class Player():
    '''
    The class Player represents a player in a poker game.
    The player has an initial stack of money and at the beginning of each round she is dealt two cards.
    Given her hand 
    '''
    
    def __init__(self, stack, blind, name):
        self.hand = hand
        self.stack = stack
        self.blind = blind
        self.name = name
        self.own_bid = 0
#        self.highest_bid = 0
        self.active = 1
        #self.actions = ['fold', 'call', ]
        
    def do(self, highest_bid, timestep, blind=None):
        if blind == 'small': # hier müssen noch ein paar Fälle abgefangen werden in denen der Spieler nicht mehr setzen kann
            self.own_bid = self.blind
            print(f'Player {self.name} has raised to ${self.blind}')
        elif blind == 'big': # hier müssen noch ein paar Fälle abgefangen werden in denen der Spieler nicht mehr setzen kann
            self.own_bid = self.blind * 2
            print(f'Player {self.name} has raised to ${self.blind*2}')
        else:
            self.own_bid == highest_bid
            
    
    def set_inactive(self):
        '''
        This function sets the player inactive if he is not willing to increase his bid to the highest bid.
        '''
        self.active = 0
        

In [261]:
import numpy as np
import itertools

class Game:
    '''
    This class takes as input:
    
    nr_of_players: the number of players that will participate in the game
    limit: the limit of the bets in each round
    blind: the size of the big blind
    stack: the size of the initial stack of the game
    
    and creates a pokergame where you can deal the cards, players can fold, call or raise
    and the game thus unfolds.
    '''

    suit = '♥ ♦ ♣ ♠'.split()
    
    # ordered strings of faces
    faces   = '2 3 4 5 6 7 8 9 10 j q k a'
    lowaces = 'a 2 3 4 5 6 7 8 9 10 j q k'
    deck = list(itertools.product(faces.split(), suit))
    #print(deck, type(deck))
    
    # faces as lists
    face   = faces.split()
    lowace = lowaces.split()
    
    def __init__(self, nr_of_players, blind, stack, limit=None):
        self.nr_of_players = nr_of_players
        self.limit = limit
        self.blind = blind
        self.stack = stack
        self.players = []
        self.player_names = list(range(nr_of_players))
        self.order = list(range(nr_of_players))
        self.position_small = 0
        self.highest_bid = 0
        self.timestep = 0
        self.game_count = 1
        self.active_players = nr_of_players
        
        for p in range(self.nr_of_players):
            player = Player(self.stack, self.blind, p+1)
            self.players.append(player)
    
    
    def deal_cards(self):
        '''
        The function deal_cards takes the list of players (self.players) and deals a hand of two cards for each player
        '''
        np.random.shuffle(deck)
        for player in self.players:
            player.hand = [deck.pop(), deck.pop()]
            assert len(player.hand) == 2
            
    
    def eliminate_players(self):
        '''
        The function eliminate_players eliminates, after each round, the players that did not call the highest bid.
        '''
        for player in self.players:
            if player.own_bid < self.highest_bid:
                player.set_inactive()
    
    
    def action_first_round(self):
        '''
        The function action calls all agents sequentially to decide on their action
        '''
        
        # create list of order in which players play
        order = [self.position_small, self.position_small+1]
        add = list(range(self.nr_of_players))
        [add.remove(x) for x in order]
        add_help = []
        
        for p in order:
            if p < self.position_small:
                add_help.append(p)
            else:
                order.append(p)
        
        [order.append(x) for x in add_help]


        # action of the small blind player 
        self.players[self.position_small].do(self.highest_bid, self.timestep, blind='small')

        # action of the big blind player
        self.players[self.position_small+1].do(self.highest_bid, self.timestep, blind='big')
        self.highest_bid = self.players[self.position_small+1].own_bid

        # create a list of players who do not have to play the blind
        # there is a mistake in here because the small blind player will not increase his bid to the big blind
        no_blind = list(range(self.nr_of_players))
        no_blind.remove(self.position_small)
        no_blind.remove(self.position_small + 1)

        for position in no_blind:
            self.player[position].do(self.highest_bid)
            if self.player[position].own_bid > self.highest_bid:
                self.highest_bid = self.player[position].own_bid

        active_bidding = 1
        while active_bidding == 1:
            for position in order:
                if self.player[position].own_bid < self.highest_bid:
                    self.player[position].do(self.highest_bid)
            
        
        
        # increase the timestep by one
        self.timestep += 1
        
        # shift the position of the small blind by one
        self.position_small += 1
        
    
    def deal_flop():
        pass
        
        
    def action_second_round(self):
        # define action for the later timesteps
        # start with small blind position and continue going forwards
        # disregards all the players that are inactive
        
        active_bidding = 1
        
        # create list of order in which players play
        order = [self.position_small, self.position_small+1]
        add = list(range(self.nr_of_players))
        [add.remove(x) for x in order]
        add_help = []
        
        for p in order:
            if p < self.position_small:
                add_help.append(p)
            else:
                order.append(p)
        
        [order.append(x) for x in add_help]
        
        # shift the position of the small blind by one
        if self.position_small < (self.nr_of_players-1):
            self.position_small += 1
        else:
            self.position_small = 0
            
        pass
    
    
    def deal_flop_2():
        pass
    
    
    def action_third_round():
        pass
    
    
    def __repr__(self):
        return '''The game is in timestep {}. The highest bid is {}. The position of the small blind is {}.\
                \nThe total money in the pot will yet have to be created. The players who are still in the game will have to be created.
                '''.format(self.timestep+1, self.highest_bid, self.position_small+1)

In [262]:
a = [1, 2, 3, 4]
b = [2, 3]
[a.remove(x) for x in b]
a

[1, 4]

In [263]:
c = []
d = []
for x in a:
    print(x)
    if x < b[-1]:
        c.append(x)
        print('has to wait')
    else:
        d.append(x)
        print('goes first')
        
[d.append(x) for x in c]

d
        

1
has to wait
4
goes first


[4, 1]

In [264]:
[b.append(x) for x in d]
b

[2, 3, 4, 1]

In [265]:
g = Game(2, 100, 1000)        

In [266]:
g

The game is in timestep 1. The highest bid is 0. The position of the small blind is 1.                
The total money in the pot will yet have to be created. The players who are still in the game will have to be created.
                

In [267]:
g.deal_cards()

In [268]:
for player in g.players:
    print(player.hand)

[('q', '♥'), ('q', '♣')]
[('6', '♥'), ('9', '♠')]


In [269]:
g.action_first_round()

Player 1 has raised to $100
Player 2 has raised to $200


In [270]:
g

The game is in timestep 2. The highest bid is 200. The position of the small blind is 2.                
The total money in the pot will yet have to be created. The players who are still in the game will have to be created.
                

In [193]:
g.nr_of_players, g.players, g.players[0]

(2,
 [<__main__.Player at 0x10f337cc0>, <__main__.Player at 0x10f337fd0>],
 <__main__.Player at 0x10f337cc0>)

In [8]:
class Game:
    playerOnesHand = []
    playerTwosHand = []
    playerOneScore = 0
    playerTwoScore = 0
    result = 0

    def __init__(self, cards):
        cards = cards.split()
        self.playerOnesHand = Hand(cards[0:5])
        self.playerTwosHand = Hand(cards[5:10])
        self.play();

    def play(self):
        self.playerOneScore = self.playerOnesHand.getScore();
        self.playerTwosCore = self.playerTwosHand.getScore();


        if (self.playerOneScore > self.playerTwoScore):
            result = 1
        else: 
            result = 2

class Hand: 
    cards = []
    values = []
    def setValues(self):
        for card in self.cards:
            self.values.append(card.value)
        self.values.sort()

        print(len(self.values))

    def isFlush(self):
        currentSuit = self.cards[0].suit
        for card in self.cards:
            if card.suit != currentSuit:
                return False
            currentSuit = card.suit
        return True

    def isRoyalFlush(self):
        for card in self.cards:
            self.values.append(card.value)

        low = min(self.values)

        if low == 10 and (low + 1 in self.values) and (low + 2 in self.values) and (low + 3 in self.values) and (low + 4 in self.values):
            return True
        else:
            return False

    def isFourOfAKind(self):
        cardOneCount = self.values.count(self.values[0])
        cardTwoCount = self.values.count(self.values[len(self.values) - 1])

        if cardOneCount == 4 | cardTwoCount == 4:
            return True
        else: 
            return False

    def isThreeOfAKind(self):
        cardOneCount = self.values.count(self.values[0])
        cardTwoCount = self.values.count(self.values[len(self.values) - 1])

        if cardOneCount == 3 | cardTwoCount == 3:
            return True
        else: 
            return False

    def isStraight(self):
        low = min(self.values)

        if low == 10 and (low + 1 in self.values) and (low + 2 in self.values) and (low + 3 in self.values) and (low + 4 in self.values):
            return True;

    def isFullHouse(self):
        self.values = []
        for card in self.cards:
            self.values.append(card.value)

        self.values.sort()
        cardOneCount = self.values.count(self.values[0])
        cardTwoCount = self.values.count(self.values[len(self.values) - 1])

        if (cardOneCount == 2 and cardTwoCount == 3 ) | (cardOneCount == 3 and cardTwoCount == 2):
            return True
        else: 
            return False

    def isTwoPair(self):
        cardOneCount = self.values.count(self.values[0])
        cardTwoCount = self.values.count(self.values[4])
        cardThreeCount = self.values.count(self.values[2])

        if(cardOneCount == 2 and cardTwoCount == 2) | (cardOneCount == 2 and cardThreeCount == 2) | (cardTwoCount == 2 and cardThreeCount == 2):
            return True;
        return False;

    def isPair(self):
        cardOneCount = self.values.count(self.values[0])
        cardTwoCount = self.values.count(self.values[4])
        cardThreeCount = self.values.count(self.values[2])

        if(cardOneCount == 2 ^ cardTwoCount == 2 ^ cardTwoCount == 3):
            return True;
        return False;

    def __init__(self, cards):
        for c in cards:
            self.cards.append(Card(c))

        self.setValues()

    def getScore(self):
        score = 0
        if self.isRoyalFlush(): score = 140000
        elif self.isFlush() and self.isStraight(): score = 130000
        elif self.isFourOfAKind(): score = 120000
        elif self.isFullHouse(): score = 110000
        elif self.isFlush(): score = 100000
        elif self.isStraight(): score = 90000
        elif self.isThreeOfAKind(): score = 80000
        elif self.isTwoPair(): score = 70000
        elif self.isTwoPair(): score = 60000
        else: score = 0  

        return score

class Card:
    # value = 0
    # suit = 0
    def __init__(self, card):
        self.value = self.determineValue(card[0])
        self.suit = card[1]

    def determineValue(x,y):
        return {
            '2':2,
            '3':3,
            '4':4,
            '5':6,
            '6':6,
            '7':7,
            '8':8,
            '9':9,
            'T':10,
            'J':11,
            'Q':12,
            'K':13,
            'A':14
        }.get(y)

    def playGames():
        player1Score = 0;
        player2Score = 0;
        
#    with open("poker-small.txt", 'w') as file:

#        for line in file:
#            game = Game(line)
#            if game.result == 1:
#                player1Score += 1
#            else:
#                player2Score += 1

    print("Player one has won " + str(player1Score) + " games! ")
    print("Player two has won " + str(player2Score) + " games! ")
    if (player1Score > player2Score):
        print("Player 1 wins!")
    else:
        print("Player 2 wins!")
 
def main():
    playGames();

if __name__ == "__main__":
    main()

NameError: name 'player1Score' is not defined

        self.result_rating = {0: {'name': 'Royal Flush',
                                  'hands': [
                                      [('A', 'hearts'), ('K', 'hearts'), ('Q', 'hearts'), ('J', 'hearts'), ('10', 'hearts')]
                                    , [('A', 'tiles'), ('K', 'tiles'), ('Q', 'tiles'), ('J', 'tiles'), ('10', 'tiles')]
                                    , [('A', 'clovers'), ('K', 'clovers'), ('Q', 'clovers'), ('J', 'clovers'), ('10', 'clovers')]
                                    , [('A', 'pikes'), ('K', 'pikes'), ('Q', 'pikes'), ('J', 'pikes'), ('10', 'pikes')]
                                 ]
                                 },
                             1: {'name': 'Straight Flush King',
                                 'hands': [
                                     [('K', 'hearts'), ('Q', 'hearts'), ('J', 'hearts'), ('10', 'hearts'), ('9', 'hearts')]
                                    , [('K', 'tiles'), ('Q', 'tiles'), ('J', 'tiles'), ('10', 'tiles'), ('9', 'tiles')]
                                    , [('K', 'clovers'), ('Q', 'clovers'), ('J', 'clovers'), ('10', 'clovers'), ('9', 'clovers')]
                                    , [('K', 'pikes'), ('Q', 'pikes'), ('J', 'pikes'), ('10', 'pikes'), ('9', 'pikes')]       
                                ]
                                },
                              2: {'name': 'Straight Flush Queen',
                                  'hands': [
                                      [('Q', 'hearts'), ('J', 'hearts'), ('10', 'hearts'), ('9', 'hearts'), ('8', 'hearts')]
                                    , [('Q', 'tiles'), ('J', 'tiles'), ('10', 'tiles'), ('9', 'tiles'), ('8', 'tiles')]
                                    , [('Q', 'clovers'), ('J', 'clovers'), ('10', 'clovers'), ('9', 'clovers'), ('8', 'clovers')]
                                    , [('Q', 'pikes'), ('J', 'pikes'), ('10', 'pikes'), ('9', 'pikes'), ('8', 'pikes')]
                                 ]
                                 },
                              3: {'name': 'Straight Flush Jack',
                                  'hands': [
                                      [('J', 'hearts'), ('10', 'hearts'), ('9', 'hearts'), ('8', 'hearts'), ('7', 'hearts')]
                                    , [('J', 'tiles'), ('10', 'tiles'), ('9', 'tiles'), ('8', 'tiles'), ('7', 'tiles')]
                                    , [('J', 'clovers'), ('10', 'clovers'), ('9', 'clovers'), ('8', 'clovers'), ('7', 'clovers')]
                                    , [('J', 'pikes'), ('10', 'pikes'), ('9', 'pikes'), ('8', 'pikes'), ('7', 'pikes')]
                                 ]
                                 },
                              4: {'name': 'Straight Flush 10',
                                  'hands': [
                                      [('10', 'hearts'), ('9', 'hearts'), ('8', 'hearts'), ('7', 'hearts'), ('6', 'hearts')]
                                    , [('10', 'tiles'), ('9', 'tiles'), ('8', 'tiles'), ('7', 'tiles'), ('6', 'tiles')]
                                    , [('10', 'clovers'), ('9', 'clovers'), ('8', 'clovers'), ('7', 'clovers'), ('6', 'clovers')]
                                    , [('10', 'pikes'), ('9', 'pikes'), ('8', 'pikes'), ('7', 'pikes'), ('6', 'pikes')]
                                 ]
                                 },
                              5: {'name': 'Straight Flush 9',
                                  'hands': [
                                      [('9', 'hearts'), ('8', 'hearts'), ('7', 'hearts'), ('6', 'hearts'), ('5', 'hearts')]
                                    , [('9', 'tiles'), ('8', 'tiles'), ('7', 'tiles'), ('6', 'tiles'), ('5', 'tiles')]
                                    , [('9', 'clovers'), ('8', 'clovers'), ('7', 'clovers'), ('6', 'clovers'), ('5', 'clovers')]
                                    , [('9', 'pikes'), ('8', 'pikes'), ('7', 'pikes'), ('6', 'pikes'), ('5', 'pikes')]
                                 ]
                                 },
                              6: {'name': 'Straight Flush 8',
                                  'hands': [
                                      [('8', 'hearts'), ('7', 'hearts'), ('6', 'hearts'), ('5', 'hearts'), ('9', 'hearts')]
                                    , [('9', 'tiles'), ('8', 'tiles'), ('7', 'tiles'), ('6', 'tiles'), ('5', 'tiles')]
                                    , [('9', 'clovers'), ('8', 'clovers'), ('7', 'clovers'), ('6', 'clovers'), ('5', 'clovers')]
                                    , [('9', 'pikes'), ('8', 'pikes'), ('7', 'pikes'), ('6', 'pikes'), ('5', 'pikes')]
                                 ]
                                 }
                             }